In [3]:
# Import modules and subroutines:
using ConfParser
using LaTeXStrings
include("../subroutines/Subroutines.jl");

3.10.9


In [4]:
# Load data and config parameters:
fid = h5open("../datasets/tnqe_data/h6_octahedron_2023-05-23.h5", "r")

conf_path = read(fid["conf_path"])

conf, op_list, gp_list, swp_list = FetchConfig("../$(conf_path)")

# Load chemical data:
cdata_list = []

for m=1:conf.nmol

    cdata = ReadIn("../"*conf.pyscf_paths[m])[1];
    push!(cdata_list, cdata)
    PrintChemData(cdata)
    
end

Molecule name: h6_0.65A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h6_octahedron/0.65.xyz
RHF energy: -2.2750088491574125
FCI energy: -2.4150646452458577
Molecule name: h6_0.75A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h6_octahedron/0.75.xyz
RHF energy: -2.4685234927711077
FCI energy: -2.623765900816072
Molecule name: h6_0.85A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h6_octahedron/0.85.xyz
RHF energy: -2.5427362470491515
FCI energy: -2.7258658998727627
Molecule name: h6_0.95A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h6_octahedron/0.95.xyz
RHF energy: -2.5522990394405527
FCI energy: -2.7708424728855965
Molecule name: h6_1.05A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h6_octahedron/1.05.xyz
RHF energy: -2.526126764176243
FCI energy: -2.7890475622976973
Molecule name: h6_1.15A
Basis set: sto-3g
Molecular geometry: ../configs/xyz_files/h6_octahedron/1.15.xyz
RHF energy: -2.4801757858099958
FCI energy: -2

In [5]:
# Collect the potential energy curves:

e_rhf_vec = [cdata_list[m].e_rhf for m=1:length(cdata_list)]
e_fci_vec = [cdata_list[m].e_fci for m=1:length(cdata_list)]

atz_labels = conf.atz_name

e_atz_vecs = []
for a=1:conf.n_atz
    e_atz_vec = [read(fid["$(conf.mol_names[m])/$(conf.atz_name[a])/E"])[1] + cdata_list[m].e_nuc for m=1:conf.nmol]
    push!(e_atz_vecs, e_atz_vec)
    display(e_atz_vec)
end

8-element Vector{Float64}:
 -2.40115055255543
 -2.6168671990041483
 -2.724106109095982
 -2.755613972479769
 -2.770472620537074
 -2.7808491228730237
 -2.7988863849659076
 -2.798105776389106

8-element Vector{Float64}:
 -2.3814609079430937
 -2.5888141675302174
 -2.6959171378123488
 -2.682749642066665
 -2.6849518640797303
 -2.6770499563274983
 -2.7834131767807726
 -2.6914675544219357

8-element Vector{Float64}:
 -2.4060492195649434
 -2.6217296469888582
 -2.7106219188705394
 -2.7595601895532047
 -2.722935271430277
 -2.7938910955968437
 -2.7990423823056982
 -2.7993730977765856

8-element Vector{Float64}:
 -2.4139745566547326
 -2.622117862624976
 -2.7206356631096247
 -2.740030089909326
 -2.7751436492699124
 -2.783554346572215
 -2.799353111423155
 -2.802364087992149

In [4]:
display(atz_labels)

4-element Vector{String}:
 "dmrg8"
 "dmrg3"
 "nomps"
 "tnqe"

In [10]:
# Plot the potential energy curves:

# x-axis information:
x_vals = [0.65,0.75,0.85,0.95,1.05,1.15,1.25,1.35]
x_label = L"r\quad(\AA)"

# Plot title:
title = "Double-dissociation of H2O in STO-3G basis"

using LaTeXStrings

plot(
    x_vals, 
    e_rhf_vec, 
    color=RGB(0.7,0.7,0.7), 
    linewidth=2.0,
    label="RHF",
    title=title,
    titlefontsize=11.0,
    titlefont=font(14,"Computer Modern"),
    xguide=x_label,
    yguide=L"E\quad(\textrm{Ha.})",
    size=(700,500)
)

plot!(
    x_vals, 
    e_fci_vec, 
    color=RGB(0.3,0.3,0.3), 
    linewidth=2.0,
    label="FCI"
)

for a=1:conf.n_atz
    
    plot!(
        x_vals, 
        e_atz_vecs[a], 
        linewidth=2.0,
        markersize=4,
        markershape=:circle,
        label=atz_labels[a]
    )
    
end


savefig("../plots/test.pdf")

"/home/oskar/Documents/TNQE/TNQE-Julia/plots/test.pdf"

In [9]:
plot!(
    bond_lengths, 
    e_dmrg8_vec, 
    color=RGB(0.4,0.4,1.0), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="DMRG; m=8"
)

plot!(
    bond_lengths, 
    e_dmrg3_vec, 
    color=RGB(0.3,0.8,0.5), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="DMRG; m=3"
)

plot!(
    bond_lengths, 
    e_nomps_vec, 
    color=color=RGB(1.0,0.8,0.3), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="Cl. NOMPS; M=5, m=3"
)

plot!(
    bond_lengths, 
    e_tnqe_vec, 
    color=color=RGB(1.0,0.4,0.4), 
    linewidth=2.0,
    markersize=4,
    markershape=:circle,
    label="TNQE (NOMPS); M=5, m=3"


LoadError: UndefVarError: e_dmrg8_vec not defined

In [9]:
parse(Float64, "1e-1")

0.1